In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
# optional
import seaborn as sns; sns.set()
import math
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import GradientBoostingClassifier
from scipy.sparse.linalg import svds
from functools import partial
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute


train = pd.read_table("../Data_competition/X_train.txt", sep=',',header=None)
test = pd.read_table("../Data_competition/X_test.txt",sep=',',header=None)
cible=pd.read_table("../Data_competition/y_train.txt",sep=',',header=None)
header=pd.read_csv("../Data_competition/header.csv",sep=',',header=None)

feature_name=header.drop(header.columns[[2]], axis=1)
feature_name=feature_name.drop(feature_name.index[[56,49,30,20,14,0,38]])[3]
feature_name
feature_name2=header.drop(header.columns[[2]], axis=1)
feature_name2=feature_name2.drop(feature_name2.index[[56,49,30,20,14,0,38]])[1]
                                 
train_via_def=train
train_via_def.columns=feature_name2
test_via_def=test
test_via_def.columns=feature_name2

frames=[train_via_def,test_via_def]
alldata= pd.concat(frames,keys=['train','test'])
alldata['x51']=alldata['x51']-2011 

Using TensorFlow backend.


In [2]:
bsc=BiScaler()

In [6]:
alldata_filled_KNN = KNN(k=5).complete(alldata)

Computing pairwise distances between 4001 samples
Computing distances for sample #1/4001, elapsed time: 0.843
Computing distances for sample #101/4001, elapsed time: 0.963
Computing distances for sample #201/4001, elapsed time: 1.108
Computing distances for sample #301/4001, elapsed time: 1.239
Computing distances for sample #401/4001, elapsed time: 1.397
Computing distances for sample #501/4001, elapsed time: 1.520
Computing distances for sample #601/4001, elapsed time: 1.667
Computing distances for sample #701/4001, elapsed time: 1.799
Computing distances for sample #801/4001, elapsed time: 1.943
Computing distances for sample #901/4001, elapsed time: 2.065
Computing distances for sample #1001/4001, elapsed time: 2.215
Computing distances for sample #1101/4001, elapsed time: 2.363
Computing distances for sample #1201/4001, elapsed time: 2.538
Computing distances for sample #1301/4001, elapsed time: 2.700
Computing distances for sample #1401/4001, elapsed time: 2.849
Computing distanc

In [14]:
alldata_filled_KNN

array([ 0.49172166,  1.26428685, -0.15857676, -0.15076807,  1.08762723,
        0.15716598, -1.05387929, -1.1169605 , -0.5164186 ,  0.26278957,
        1.04783908,  0.94763389,  0.60168794, -0.23575826,  0.17582961,
        0.39246612,  0.1078882 ,  0.01666201, -0.04175543, -0.04732615,
       -0.39925639,  0.02492688,  0.22150317, -0.01160996, -0.13586108,
        0.28304116, -0.11541101,  0.16440892, -0.39185438,  0.34897147,
        0.06626888,  0.61581002,  1.42047292, -0.37906425,  0.37902754,
        0.11845125,  0.43864351, -0.26293731, -0.1378807 ,  0.18950561,
        0.88755757, -0.10337045, -0.40572264, -0.99622344, -0.03962045,
        1.07679487,  5.2844729 ,  0.19530428, -0.37605104, -0.61527922,  0.        ])

In [33]:
colonnenames=['x'+str(j) for j in range(1,52)]

In [34]:
datacomplete=pd.DataFrame(alldata_filled_KNN)
datacomplete.index=alldata.index

In [35]:
datacomplete.columns=colonnenames

In [36]:
datacomplete.head()

x1        x2        x3        x4        x5        x6        x7  \
train 0  0.491722  1.264287 -0.158577 -0.150768  1.087627  0.157166 -1.053879   
      1 -0.249351  0.046088 -0.040435  0.105421 -1.098919  0.289237 -0.338154   
      2  0.986824 -0.189678  0.059693  0.010147  0.777260 -1.051633 -1.011747   
      3 -0.265990 -0.517960 -0.114526 -0.073818  1.889234 -0.161217 -0.947844   
      4  0.199900 -0.095447  0.263825  0.040790 -1.125166 -1.318912 -0.887661   

               x8        x9       x10       x11       x12       x13       x14  \
train 0 -1.116961 -0.516419  0.262790  1.047839  0.947634  0.601688 -0.235758   
      1  0.129297  0.807200 -0.594330 -0.581096  0.013371  0.195210  0.010919   
      2 -0.756690 -0.696582  0.874877 -0.273440  0.086508  0.688904  0.011214   
      3 -0.650268 -0.712451  1.178903  0.126789  0.596843  0.788205  0.131752   
      4 -1.202686 -0.304145 -0.413741 -0.806276  0.415610  0.060390  0.010513   

              x15       x16       x17       x18       x19       x20       x21  \
train 0  0.175830  0.392466  0.107888  0.016662 -0.041755 -0.047326 -0.399256   
      1 -0.052029 -0.467408 -0.179635  0.074411 -0.105522  0.008015 -0.736507   
      2  0.182287  0.351119 -0.327590  0.026406  0.154479  0.290977  0.956582   
      3  0.423003  0.150610 -0.223991  0.030660 -0.081760  0.246647  0.044999   
      4  0.302067 -0.036239 -0.290726  0.037814 -0.541424  0.204252 -1.118377   

              x22       x23       x24       x25       x26       x27       x28  \
train 0  0.024927  0.221503 -0.011610 -0.135861  0.283041 -0.115411  0.164409   
      1 -0.068438 -0.417189  0.087101 -0.362549  0.270641 -0.657171  0.351780   
      2  0.388028  0.191073  0.086748  1.195586  0.310467  0.202824  0.131217   
      3  0.285686 -0.455399 -0.077836  0.111350  0.271222  0.286065  0.310761   
      4 -0.014610 -0.012254  2.624526  0.983099 -1.652295 -1.170604  0.388641   

              x29       x30       x31       x32       x33       x34       x35  \
train 0 -0.391854  0.348971  0.066269  0.615810  1.420473 -0.379064  0.379028   
      1  0.510574  0.568804  0.248886  0.445685  0.169736  0.051006  0.089067   
      2 -0.870593  0.348219  0.286557  0.452968  0.402763 -1.309222  0.406656   
      3 -0.703435  0.465727  0.240896  0.822045  0.497331 -1.189501  0.242710   
      4  1.233810  0.349228 -0.061911 -2.776458 -0.118833  0.330532  0.168075   

              x36       x37       x38       x39       x40       x41       x42  \
train 0  0.118451  0.438644 -0.262937 -0.137881  0.189506  0.887558 -0.103370   
      1 -0.171755 -0.329228 -0.203004 -0.413300  0.225627  0.049266  0.468655   
      2  0.296849  0.264740  0.427946  0.513461  0.320219  0.404907 -0.271227   
      3  0.440182  0.373681 -0.184058  0.686641  0.117178  0.430068 -0.108483   
      4 -1.209992  0.299191  0.033525  0.040870  0.393697 -0.424390 -0.568337   

              x43       x44       x45       x46       x47       x48       x49  \
train 0 -0.405723 -0.996223 -0.039620  1.076795  5.284473  0.195304 -0.376051   
      1  0.083019  0.527339 -0.291376 -0.165093  0.652600  1.096701  0.658806   
      2 -0.282350  0.995816  0.004617  0.480180 -0.518204 -0.105363 -0.541989   
      3 -0.378151  0.219661 -0.431705  0.743919 -0.216563  0.479940 -0.755457   
      4 -0.180425  0.887360 -0.686345 -0.699968 -0.355482  0.179848 -0.241917   

              x50  x51  
train 0 -0.615279  0.0  
      1  0.577559  0.0  
      2 -0.461882  0.0  
      3 -0.526293  0.0  
      4 -0.535741  0.0

In [38]:
test_nan_KNN=datacomplete.ix['test']
train_nan_KNN=datacomplete.ix['train']

In [39]:
tuned_parameters = [{'loss': ['deviance','exponential'],
                    'max_features':[5,10,20,50],
                    'max_depth':[3,5,10,30],
                    'n_estimators':[100,300]}]

clf = GridSearchCV(GradientBoostingClassifier(), tuned_parameters, cv=10,verbose=1)
clf.fit(train_nan_KNN, cible[0].values)


print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()

Fitting 10 folds for each of 64 candidates, totalling 640 fits


[Parallel(n_jobs=1)]: Done 640 out of 640 | elapsed: 18.9min finished


Best parameters set found on development set:

{'max_features': 20, 'n_estimators': 300, 'loss': 'deviance', 'max_depth': 5}

Grid scores on development set:

0.958 (+/-0.037) for {'max_features': 5, 'n_estimators': 100, 'loss': 'deviance', 'max_depth': 3}
0.958 (+/-0.027) for {'max_features': 5, 'n_estimators': 300, 'loss': 'deviance', 'max_depth': 3}
0.956 (+/-0.037) for {'max_features': 10, 'n_estimators': 100, 'loss': 'deviance', 'max_depth': 3}
0.962 (+/-0.031) for {'max_features': 10, 'n_estimators': 300, 'loss': 'deviance', 'max_depth': 3}
0.958 (+/-0.028) for {'max_features': 20, 'n_estimators': 100, 'loss': 'deviance', 'max_depth': 3}
0.962 (+/-0.028) for {'max_features': 20, 'n_estimators': 300, 'loss': 'deviance', 'max_depth': 3}
0.957 (+/-0.028) for {'max_features': 50, 'n_estimators': 100, 'loss': 'deviance', 'max_depth': 3}
0.957 (+/-0.033) for {'max_features': 50, 'n_estimators': 300, 'loss': 'deviance', 'max_depth': 3}
0.958 (+/-0.035) for {'max_features': 5, 'n_estimat

In [40]:
clf.best_score_

0.96375

In [41]:
clf.best_params_

{'loss': 'deviance', 'max_depth': 5, 'max_features': 20, 'n_estimators': 300}

In [48]:
clf=GradientBoostingClassifier(loss='deviance',max_depth=5,max_features=20,n_estimators=300)
clf.fit(train_nan_KNN, cible[0].values)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=20, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=300, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [43]:
y_pred_clf=clf.predict(test_nan_KNN)

index=range(len(y_pred_clf))
data=np.transpose([index,y_pred_clf])
a = pd.DataFrame(data=data,columns=['Id','Prediction'])
a.to_csv("submission9.csv", sep=",",index=False)

### Bagging classifier

In [49]:
from sklearn.ensemble import BaggingClassifier

In [61]:
clf=GradientBoostingClassifier(loss='deviance',max_depth=5,max_features=20,n_estimators=300)

In [83]:
BC=BaggingClassifier(base_estimator=clf,n_estimators=50,oob_score=True)

In [84]:
BC.fit(train_nan_KNN, cible[0].values)

BaggingClassifier(base_estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=20, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=300, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=1, oob_score=True,
         random_state=None, verbose=0, warm_start=False)

In [85]:
BC.oob_score_

0.96083333333333332

In [76]:
y_pred_clf=BC.predict(test_nan_KNN)

index=range(len(y_pred_clf))
data=np.transpose([index,y_pred_clf])
a = pd.DataFrame(data=data,columns=['Id','Prediction'])
a.to_csv("submission10.csv", sep=",",index=False)

### Diminuer/augmenter knn

In [44]:
alldata_filled_KNN = KNN(k=9).complete(alldata)
alldata_filled_KNN
colonnenames=['x'+str(j) for j in range(1,52)]
datacomplete=pd.DataFrame(alldata_filled_KNN)
datacomplete.index=alldata.index
datacomplete.columns=colonnenames
test_nan_KNN=datacomplete.ix['test']
train_nan_KNN=datacomplete.ix['train']
tuned_parameters = [{'loss': ['deviance','exponential'],
                    'max_features':[5,10,20,50],
                    'max_depth':[3,5,10,30],
                    'n_estimators':[100,300]}]

clf = GridSearchCV(GradientBoostingClassifier(), tuned_parameters, cv=10,verbose=1)
clf.fit(train_nan_KNN, cible[0].values)


print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()

Computing pairwise distances between 4001 samples
Computing distances for sample #1/4001, elapsed time: 0.750
Computing distances for sample #101/4001, elapsed time: 0.889
Computing distances for sample #201/4001, elapsed time: 1.027
Computing distances for sample #301/4001, elapsed time: 1.185
Computing distances for sample #401/4001, elapsed time: 1.339
Computing distances for sample #501/4001, elapsed time: 1.463
Computing distances for sample #601/4001, elapsed time: 1.604
Computing distances for sample #701/4001, elapsed time: 1.741
Computing distances for sample #801/4001, elapsed time: 1.883
Computing distances for sample #901/4001, elapsed time: 2.010
Computing distances for sample #1001/4001, elapsed time: 2.186
Computing distances for sample #1101/4001, elapsed time: 2.352
Computing distances for sample #1201/4001, elapsed time: 2.506
Computing distances for sample #1301/4001, elapsed time: 2.648
Computing distances for sample #1401/4001, elapsed time: 2.819
Computing distanc

KeyboardInterrupt: 